In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor

In [2]:
market = Market()
market.connect()
dates = market.retrieve_date_range()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
stuff = pd.to_datetime(dates["date"])

In [4]:
start = (stuff.max() + timedelta(days=1)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2021-12-03 2021-12-08


In [5]:
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        data = TiingoExtractor.extract(ticker,start,end)
        data["ticker"] = ticker
        market.store("prices",data)
    except Exception as e:
        print(str(e))
market.disconnect()

 16%|█████████████████████▍                                                                                                              | 82/505 [00:14<01:11,  5.88it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


 61%|███████████████████████████████████████████████████████████████████████████████▋                                                   | 307/505 [00:51<00:40,  4.86it/s]

project_market prices documents must be a non-empty list


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 503/505 [01:23<00:00,  7.39it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment
Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [01:23<00:00,  6.02it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment
Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


In [6]:
data

In [7]:
analysis = []
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_data(ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        periods = ["year","quarter","month","week"]
        ep = test.tail(1)["adjClose"].item()
        # periods = ["year"]
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(str(e))
market.disconnect()

 13%|████████████████▉                                                                                                                   | 65/505 [02:03<13:50,  1.89s/it]

'date'


 16%|████████████████████▋                                                                                                               | 79/505 [02:30<13:15,  1.87s/it]

'date'


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 502/505 [16:05<00:05,  1.87s/it]

...

'date'


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 504/505 [16:08<00:01,  1.86s/it]

'date'


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [16:10<00:00,  1.92s/it]

'date'


In [8]:
a = pd.DataFrame(analysis).merge(sp5.rename(columns={"Symbol":"ticker"})[["ticker","Security","GICS Sector"]],on="ticker",how="left")
a.to_csv("analysis.csv")

In [9]:
period = "day"
a.sort_values(f"{period}_gain",ascending=False).head(50)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector
196,1.179445,0.064564,0.039245,0.086290,0.086290,FTNT,Fortinet,Information Technology
345,1.474597,0.563350,0.031557,0.079569,0.079569,NVDA,Nvidia,Information Technology
55,0.714309,0.210066,0.119956,0.076373,0.076373,AZO,AutoZone,Consumer Discretionary
413,0.259211,0.048341,0.106443,0.075989,0.075989,NOW,ServiceNow,Information Technology
141,1.400106,0.169229,0.122233,0.068003,0.068003,FANG,Diamondback Energy,Energy
347,0.486878,0.238530,0.071300,0.065186,0.065186,NXPI,NXP,Information Technology
139,1.954799,0.225754,0.104799,0.065156,0.065156,DVN,Devon Energy,Energy
140,0.560910,0.015420,0.018328,0.064775,0.064775,DXCM,DexCom,Health Care
45,0.815714,0.214977,0.034417,0.064527,0.064527,AMAT,Applied Materials,Information Technology
315,1.527074,-0.172213,-0.090982,0.064147,0.064147,MRNA,Moderna,Health Care


In [10]:
a.groupby("GICS Sector").mean().sort_values("day_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Energy,0.642056,0.054228,0.071036,0.031817,0.032449
Information Technology,0.267948,0.074554,0.044595,0.030182,0.030182
Financials,0.372649,0.027361,0.052041,0.017442,0.017442
Health Care,0.174940,0.006788,0.036604,0.017237,0.017237
Consumer Discretionary,0.298089,0.067243,0.063637,0.017102,0.017102
Real Estate,0.473677,0.082459,0.060457,0.015391,0.015391
Materials,0.278143,0.069907,0.052363,0.015168,0.015168
Industrials,0.274022,0.058197,0.055634,0.010509,0.010509
Utilities,0.138527,0.063121,0.050921,0.006925,0.006925
